In [1]:
import pandas as pd

Considera la siguiente situación:

* Una organización dispone del registro de un conjunto de transacciones, las cuales están compuestas por conjuntos de artículos/productos que adquieren sus clientes. 

* Sobre dichas transacciones se busca identificar conjuntos de artículos que sean adquiridos **juntos** con mucha frecuencia, de manera que podamos hacer predicciones del tipo: 

 $$\color{blue}{\text{Si un cliente adquiere los productos del conjunto} X, \text{ entonces es muy probable que también adquiera los productos del conjunto }  Y }$$

Un ejemplo está compuesto por la **cerveza** y los **pañales**:

https://www.esan.edu.pe/conexion/actualidad/2016/07/12/panales-y-cerveza/


En este documento exploraremos un conjunto de transacciones de ejemplo para revisar los conceptos principales del análisis de la canasta de compras:

In [2]:
# leemos un conjunto de transacciones

dat = pd.read_csv("transactions.csv")
print(dat.values.shape) #48 filas y 2 columnas

(48, 2)


In [3]:
dat.head(n=10)

,Customer,Product
0,1,Bread
1,1,Fruit
2,1,Jelly
3,1,Milk
4,1,PeanutButter
5,2,Bread
6,2,Fruit
7,2,Jelly
8,2,Milk
9,2,PeanutButter


Revisemos algunas definiciones básicas del análisis de la canasta de compras.

# Definiciones

## Itemset (artículos)

Es el conjunto de **artículos** de interés:

$$I = \{i_1, i_2, \ldots, i_m\}$$


In [4]:
# para nuestro conjunto de datos de ejemplo:

I = set(dat["Product"])
I #los artículos están ordendos alfabéticamente

{'Beer',
 'Bread',
 'Cheese',
 'ChocolateSauce',
 'Fruit',
 'Jelly',
 'Milk',
 'PeanutButter',
 'PotatoChips',
 'Soda',
 'Steak',
 'Vegetables',
 'WhippedCream',
 'Yogurt'}

In [5]:
len(I)

14

## Transacciones

Es el registro de cada compra que realizan los clientes:

$$T = (t_1, t_2, t_3, \ldots, t_n)$$

observa que cada transacción es un subconjunto del conjunto de artículos:

$$t_i \subseteq I, \forall i$$


Transformamos el conjunto de datos para identificar los artículos de cada transacción más claramente:

In [6]:
T = dat.groupby(['Customer']).agg({'Product': set})
T

,Product
Customer,
1,"{Milk, PeanutButter, Bread, Fruit, Jelly}"
2,"{Milk, PeanutButter, PotatoChips, Bread, Fruit..."
3,"{WhippedCream, Fruit, Beer, ChocolateSauce}"
4,"{Steak, PotatoChips, Bread, Fruit, Jelly, Soda}"
5,"{Milk, PeanutButter, Fruit, Jelly, Soda}"
6,"{Milk, PotatoChips, Bread, Fruit, Jelly, Soda}"
7,"{PotatoChips, Fruit, Milk, Soda}"
8,"{Fruit, Milk, PeanutButter, Soda}"
9,"{Cheese, Fruit, Yogurt}"


In [7]:
# renombramos la columna
T = T.rename(columns={"Product": "Transaction"})
T

,Transaction
Customer,
1,"{Milk, PeanutButter, Bread, Fruit, Jelly}"
2,"{Milk, PeanutButter, PotatoChips, Bread, Fruit..."
3,"{WhippedCream, Fruit, Beer, ChocolateSauce}"
4,"{Steak, PotatoChips, Bread, Fruit, Jelly, Soda}"
5,"{Milk, PeanutButter, Fruit, Jelly, Soda}"
6,"{Milk, PotatoChips, Bread, Fruit, Jelly, Soda}"
7,"{PotatoChips, Fruit, Milk, Soda}"
8,"{Fruit, Milk, PeanutButter, Soda}"
9,"{Cheese, Fruit, Yogurt}"


## Conteo

Para un suconjunto de artículos, $X = \{a_1, a_2, \ldots, a_x\}$, el **conteo** es el número transacciones que contienen al conjunto $X$

$$\color{blue}{X.\text{count}:} \text{Total de transacciones que contienen al conjunto } X$$


In [8]:
X = set(["Bread", "Milk"])

In [9]:
X_subset =  T["Transaction"].apply(lambda t: X.issubset(t))
X_subset

Customer
1      True
2      True
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10    False
Name: Transaction, dtype: bool

In [10]:
sum(X_subset)

3

Dado que aplicaremos este proceso múltiples veces vale la pena definirlo en una función:

In [11]:
def conteo(X, T):
    X_subset =  T["Transaction"].apply(lambda t: X.issubset(t))
    return sum(X_subset)

In [12]:
#conteo(set(["Milk", "Soda"]), T)
conteo(set(["Bread", "Jelly"]), T)

4

## Regla de asociación

Una regla de asociación se establece entre dos conjuntos de artículos diferentes, $X$ y $Y$. 

Se simboliza:  

$$X \rightarrow Y$$

y refleja la siguiente afirmación: **es suficiente con conocer que el cliente adquiere el conjunto de artículos $X$ para predecir que adquirirá el conjunto de artículos $Y$**.

Un ejemplo es la siguiente regla:

$$\{\text{Soda, Fruit}\} \rightarrow \{\text{Milk}\}$$

Llamamos a $X$ **antecedente** de la regla y a $Y$ el **consecuente**.


## Soporte de una regla

Para la regla $X \rightarrow Y$ definimos el soporte como:

$$\text{support}_{X\rightarrow Y} = \frac{(X\cup Y).\text{count}}{n}$$

siendo $n$ el número de transacciones.

In [13]:
def soporte(X,Y,T):
    XuY = X.union(Y)
    return conteo(XuY, T)/len(T)

In [14]:
#Soporte de la regla "Milk, Bread" ----> "Jelly"

X = set(["Soda", "Fruit"])
Y = set(["Milk"])

soporte(X,Y,T)

0.5

## Confianza 

La confianza de una regla se define 

$$\text{confidence}_{X\rightarrow Y} = \frac{(X\cup Y).\text{count}}{X.\text{count}}$$

In [15]:
def confianza(X,Y,T):
    XuY = X.union(Y)
    return conteo(XuY, T)/conteo(X, T)

Como ejemplo, evaluamos la confianza de la regla:

$$\{\text{Milk}\} \rightarrow \{\text{Soda}\}$$

In [16]:
confianza(set(["Milk"]), set(["Soda"]), T)

0.8333333333333334

In [17]:
confianza(set(["Milk", "Bread"]), set(["Soda"]), T)
#confianza(set(["Soda"]), set(["Milk", "Bread"]), T) #la confianza no es conmutativa

0.6666666666666666

# Objetivo


Identificar **todas** las reglas, $X\rightarrow Y$, tales que 

$$\color{blue}{\text{Soporte}(X\rightarrow Y) \geq \text{supp}_{min}}$$

Para lograr este objetivo, necesitamos primero encontrar conjuntos de artículos cuyo **soporte** supere el umbral mínimo $\text{supp}_{min}$, a estos conjuntos los llamamos **conjuntos frecuentes**.

Para ejemplificar, utilizaremos una estrategia denominada **fuerza bruta**. Iniciamos identificando a los conjuntos frecuentes de **un solo elemento**. 

Considera un soporte mínimo de $0.3$

In [18]:
from itertools import combinations

In [19]:
Items = list(I)

Formamos todos los subconjuntos con un solo artículo.

In [20]:
#subconjuntos de cardinalidad fija
sets_1 = combinations(Items,1) #jugar con la cardinalidad

for s in sets_1:
    X = set([s[0]]) 
    print(X)

{'Yogurt'}
{'Milk'}
{'PeanutButter'}
{'Beer'}
{'Steak'}
{'ChocolateSauce'}
{'Cheese'}
{'PotatoChips'}
{'Bread'}
{'Fruit'}
{'Vegetables'}
{'Jelly'}
{'Soda'}
{'WhippedCream'}


Filtramos aquellos que superen el soporte

In [21]:
#subconjuntos de un solo elemento con soporte mayor al mínimo
sets = combinations(Items,1) #jugar con la cardinalidad

supp_min = 0.3

for s in sets:
    X = set([s[0]]) 
    supp = soporte(X,X,T)
    if supp >= supp_min:
        print(X)

{'Milk'}
{'PeanutButter'}
{'PotatoChips'}
{'Bread'}
{'Fruit'}
{'Jelly'}
{'Soda'}


Repetimos el proceso, ahora con subconjuntos de dos artículos.

In [22]:
sets = combinations(Items,2) 

for s in sets:
    X = set(s) 
    supp = soporte(X,X,T)
    if supp >= supp_min:
        print(sorted(X))

['Milk', 'PeanutButter']
['Milk', 'PotatoChips']
['Bread', 'Milk']
['Fruit', 'Milk']
['Jelly', 'Milk']
['Milk', 'Soda']
['Fruit', 'PeanutButter']
['Jelly', 'PeanutButter']
['PeanutButter', 'Soda']
['Bread', 'PotatoChips']
['Fruit', 'PotatoChips']
['Jelly', 'PotatoChips']
['PotatoChips', 'Soda']
['Bread', 'Fruit']
['Bread', 'Jelly']
['Bread', 'Soda']
['Fruit', 'Jelly']
['Fruit', 'Soda']
['Jelly', 'Soda']


<font color="red">Observa que todos los artículos que aparecen en algún subconjunto de 2 también fue filtrado como conjunto frecuente de 1 artículo.</font>

In [23]:
sets = combinations(Items,3) 

for s in sets:
    X = set(s) 
    supp = soporte(X,X,T)
    if supp >= supp_min:
        print(sorted(X))

['Fruit', 'Milk', 'PeanutButter']
['Jelly', 'Milk', 'PeanutButter']
['Milk', 'PeanutButter', 'Soda']
['Fruit', 'Milk', 'PotatoChips']
['Milk', 'PotatoChips', 'Soda']
['Bread', 'Fruit', 'Milk']
['Bread', 'Jelly', 'Milk']
['Fruit', 'Jelly', 'Milk']
['Fruit', 'Milk', 'Soda']
['Jelly', 'Milk', 'Soda']
['Fruit', 'Jelly', 'PeanutButter']
['Fruit', 'PeanutButter', 'Soda']
['Bread', 'Fruit', 'PotatoChips']
['Bread', 'Jelly', 'PotatoChips']
['Bread', 'PotatoChips', 'Soda']
['Fruit', 'Jelly', 'PotatoChips']
['Fruit', 'PotatoChips', 'Soda']
['Jelly', 'PotatoChips', 'Soda']
['Bread', 'Fruit', 'Jelly']
['Bread', 'Fruit', 'Soda']
['Bread', 'Jelly', 'Soda']
['Fruit', 'Jelly', 'Soda']


In [24]:
sets = combinations(Items,4) 

for s in sets:
    X = set(s) 
    supp = soporte(X,X,T)
    if supp >= supp_min:
        print(sorted(X))

['Fruit', 'Jelly', 'Milk', 'PeanutButter']
['Fruit', 'Milk', 'PeanutButter', 'Soda']
['Fruit', 'Milk', 'PotatoChips', 'Soda']
['Bread', 'Fruit', 'Jelly', 'Milk']
['Fruit', 'Jelly', 'Milk', 'Soda']
['Bread', 'Fruit', 'Jelly', 'PotatoChips']
['Bread', 'Fruit', 'PotatoChips', 'Soda']
['Bread', 'Jelly', 'PotatoChips', 'Soda']
['Fruit', 'Jelly', 'PotatoChips', 'Soda']
['Bread', 'Fruit', 'Jelly', 'Soda']


In [25]:
sets = combinations(Items,5) 

for s in sets:
    X = set(s) 
    supp = soporte(X,X,T)
    if supp >= supp_min:
        print(sorted(X))

['Bread', 'Fruit', 'Jelly', 'PotatoChips', 'Soda']


In [26]:
sets = combinations(Items,6) 

for s in sets:
    X = set(s) 
    supp = soporte(X,X,T)
    if supp >= supp_min:
        print(sorted(X))

Revisar información del sitio: 

https://www.analyticsvidhya.com/blog/2021/10/a-comprehensive-guide-on-market-basket-analysis/

# Algoritmo A-Priori

## Conjuntos frecuentes (un solo elemento)

In [27]:
#subconjuntos de un solo elemento con soporte mayor al mínimo
sets = combinations(Items,1) #jugar con la cardinalidad

supp_min = 0.3

F1 = []

for s in sets:
    X = set([s[0]]) 
    supp = soporte(X,X,T)
    if supp >= supp_min:
        F1.append(X)
F1

[{'Milk'},
 {'PeanutButter'},
 {'PotatoChips'},
 {'Bread'},
 {'Fruit'},
 {'Jelly'},
 {'Soda'}]

## Conjuntos frecuentes (dos elementos)

**Candidatos**

In [28]:
C2 = []

for i,s in enumerate(F1):
    for j,t in enumerate(F1):
        if j > i:
            sUt = s.union(t)
            print(sorted(sUt))
            C2.append(set(sUt))

['Milk', 'PeanutButter']
['Milk', 'PotatoChips']
['Bread', 'Milk']
['Fruit', 'Milk']
['Jelly', 'Milk']
['Milk', 'Soda']
['PeanutButter', 'PotatoChips']
['Bread', 'PeanutButter']
['Fruit', 'PeanutButter']
['Jelly', 'PeanutButter']
['PeanutButter', 'Soda']
['Bread', 'PotatoChips']
['Fruit', 'PotatoChips']
['Jelly', 'PotatoChips']
['PotatoChips', 'Soda']
['Bread', 'Fruit']
['Bread', 'Jelly']
['Bread', 'Soda']
['Fruit', 'Jelly']
['Fruit', 'Soda']
['Jelly', 'Soda']


Al formar los conjuntos de candidatos uniendo solamente conjuntos frecuentes, limitamos la búsqueda solamente a 21 conjuntos: 

In [29]:
len(C2)

21

Nota que si hubiéramos utilizado fuerza bruta se hubieran tenido que formar:

$$\binom{14}{2} = \frac{14\times 13}{2} = 91$$

subconjuntos.

In [30]:
14*13/2

91.0

**Verificamos cuáles candidatos tienen suficiente soporte.**

In [31]:
#subconjuntos de dos elementos con soporte mayor al mínimo

F2 = []

for s in C2:
    supp = soporte(s,s,T)
    if supp >= supp_min:
        F2.append(s)
F2

[{'Milk', 'PeanutButter'},
 {'Milk', 'PotatoChips'},
 {'Bread', 'Milk'},
 {'Fruit', 'Milk'},
 {'Jelly', 'Milk'},
 {'Milk', 'Soda'},
 {'Fruit', 'PeanutButter'},
 {'Jelly', 'PeanutButter'},
 {'PeanutButter', 'Soda'},
 {'Bread', 'PotatoChips'},
 {'Fruit', 'PotatoChips'},
 {'Jelly', 'PotatoChips'},
 {'PotatoChips', 'Soda'},
 {'Bread', 'Fruit'},
 {'Bread', 'Jelly'},
 {'Bread', 'Soda'},
 {'Fruit', 'Jelly'},
 {'Fruit', 'Soda'},
 {'Jelly', 'Soda'}]

In [32]:
len(F2)

19

## Conjuntos frecuentes (tres elementos)

**Candidatos**

Formamos los candidatos de tres elementos a partir del conjunto frecuente $F_2$

In [33]:
C3 = []

for i,s in enumerate(F2):
    s_pre = set(sorted(s)[:-1])
    for j,t in enumerate(F2):
        if j > i:
            t_pre = set(sorted(t)[:-1])
            if t_pre.issubset(s_pre) and s_pre.issubset(t_pre):
                sUt = s.union(t)
                print(sorted(sUt))
                C3.append(set(sUt))

['Milk', 'PeanutButter', 'PotatoChips']
['Milk', 'PeanutButter', 'Soda']
['Milk', 'PotatoChips', 'Soda']
['Bread', 'Milk', 'PotatoChips']
['Bread', 'Fruit', 'Milk']
['Bread', 'Jelly', 'Milk']
['Bread', 'Milk', 'Soda']
['Fruit', 'Milk', 'PeanutButter']
['Fruit', 'Milk', 'PotatoChips']
['Fruit', 'Jelly', 'Milk']
['Fruit', 'Milk', 'Soda']
['Jelly', 'Milk', 'PeanutButter']
['Jelly', 'Milk', 'PotatoChips']
['Jelly', 'Milk', 'Soda']
['Fruit', 'PeanutButter', 'PotatoChips']
['Fruit', 'Jelly', 'PeanutButter']
['Fruit', 'PeanutButter', 'Soda']
['Jelly', 'PeanutButter', 'PotatoChips']
['Jelly', 'PeanutButter', 'Soda']
['Bread', 'Fruit', 'PotatoChips']
['Bread', 'Jelly', 'PotatoChips']
['Bread', 'PotatoChips', 'Soda']
['Fruit', 'Jelly', 'PotatoChips']
['Fruit', 'PotatoChips', 'Soda']
['Jelly', 'PotatoChips', 'Soda']
['Bread', 'Fruit', 'Jelly']
['Bread', 'Fruit', 'Soda']
['Bread', 'Jelly', 'Soda']
['Fruit', 'Jelly', 'Soda']


In [34]:
len(C3)

29

Mientras que por fuerza bruta se hubieran tenido que revisar 364 subconjuntos:

In [35]:
#  14!/(11!3!) = 14*13*12/6

14*13*12/6

364.0

Filtramos los candidatos con suficiente soporte.

In [36]:
#subconjuntos de tres elementos con soporte mayor al mínimo

F3 = []

for s in C3:
    supp = soporte(s,s,T)
    if supp >= supp_min:
        F3.append(s)
F3

[{'Milk', 'PeanutButter', 'Soda'},
 {'Milk', 'PotatoChips', 'Soda'},
 {'Bread', 'Fruit', 'Milk'},
 {'Bread', 'Jelly', 'Milk'},
 {'Fruit', 'Milk', 'PeanutButter'},
 {'Fruit', 'Milk', 'PotatoChips'},
 {'Fruit', 'Jelly', 'Milk'},
 {'Fruit', 'Milk', 'Soda'},
 {'Jelly', 'Milk', 'PeanutButter'},
 {'Jelly', 'Milk', 'Soda'},
 {'Fruit', 'Jelly', 'PeanutButter'},
 {'Fruit', 'PeanutButter', 'Soda'},
 {'Bread', 'Fruit', 'PotatoChips'},
 {'Bread', 'Jelly', 'PotatoChips'},
 {'Bread', 'PotatoChips', 'Soda'},
 {'Fruit', 'Jelly', 'PotatoChips'},
 {'Fruit', 'PotatoChips', 'Soda'},
 {'Jelly', 'PotatoChips', 'Soda'},
 {'Bread', 'Fruit', 'Jelly'},
 {'Bread', 'Fruit', 'Soda'},
 {'Bread', 'Jelly', 'Soda'},
 {'Fruit', 'Jelly', 'Soda'}]

In [37]:
len(F3)

22

## Conjuntos frecuentes (4 elementos)

In [38]:
C4 = []

for i,s in enumerate(F3):
    s_pre = set(sorted(s)[:-1])
    for j,t in enumerate(F3):
        if j > i:
            t_pre = set(sorted(t)[:-1])
            if t_pre.issubset(s_pre) and s_pre.issubset(t_pre):
                sUt = s.union(t)
                print(sorted(sUt))
                C4.append(set(sUt))

['Bread', 'Fruit', 'Milk', 'PotatoChips']
['Bread', 'Fruit', 'Jelly', 'Milk']
['Bread', 'Fruit', 'Milk', 'Soda']
['Bread', 'Jelly', 'Milk', 'PotatoChips']
['Bread', 'Jelly', 'Milk', 'Soda']
['Fruit', 'Milk', 'PeanutButter', 'PotatoChips']
['Fruit', 'Milk', 'PeanutButter', 'Soda']
['Fruit', 'Milk', 'PotatoChips', 'Soda']
['Fruit', 'Jelly', 'Milk', 'PeanutButter']
['Fruit', 'Jelly', 'Milk', 'PotatoChips']
['Fruit', 'Jelly', 'Milk', 'Soda']
['Jelly', 'Milk', 'PeanutButter', 'Soda']
['Fruit', 'Jelly', 'PeanutButter', 'PotatoChips']
['Fruit', 'Jelly', 'PeanutButter', 'Soda']
['Bread', 'Fruit', 'Jelly', 'PotatoChips']
['Bread', 'Fruit', 'PotatoChips', 'Soda']
['Bread', 'Jelly', 'PotatoChips', 'Soda']
['Fruit', 'Jelly', 'PotatoChips', 'Soda']
['Bread', 'Fruit', 'Jelly', 'Soda']


In [39]:
len(C4)

19

In [40]:
14*13*12*11/24

1001.0

In [41]:
#subconjuntos de cuatro elementos con soporte mayor al mínimo

F4 = []

for s in C4:
    supp = soporte(s,s,T)
    if supp >= supp_min:
        F4.append(s)
F4

[{'Bread', 'Fruit', 'Jelly', 'Milk'},
 {'Fruit', 'Milk', 'PeanutButter', 'Soda'},
 {'Fruit', 'Milk', 'PotatoChips', 'Soda'},
 {'Fruit', 'Jelly', 'Milk', 'PeanutButter'},
 {'Fruit', 'Jelly', 'Milk', 'Soda'},
 {'Bread', 'Fruit', 'Jelly', 'PotatoChips'},
 {'Bread', 'Fruit', 'PotatoChips', 'Soda'},
 {'Bread', 'Jelly', 'PotatoChips', 'Soda'},
 {'Fruit', 'Jelly', 'PotatoChips', 'Soda'},
 {'Bread', 'Fruit', 'Jelly', 'Soda'}]

## Conjuntos frecuentes (5 elementos)

In [42]:
C5 = []

for i,s in enumerate(F4):
    s_pre = set(sorted(s)[:-1])
    for j,t in enumerate(F4):
        if j > i:
            t_pre = set(sorted(t)[:-1])
            if t_pre.issubset(s_pre) and s_pre.issubset(t_pre):
                sUt = s.union(t)
                print(sorted(sUt))
                C5.append(set(sUt))

['Bread', 'Fruit', 'Jelly', 'Milk', 'PotatoChips']
['Bread', 'Fruit', 'Jelly', 'Milk', 'Soda']
['Fruit', 'Jelly', 'Milk', 'PeanutButter', 'Soda']
['Bread', 'Fruit', 'Jelly', 'PotatoChips', 'Soda']


In [43]:
14*13*12*11*10/120

2002.0

In [44]:
#subconjuntos de cinco elementos con soporte mayor al mínimo

F5 = []

for s in C5:
    supp = soporte(s,s,T)
    if supp >= supp_min:
        F5.append(s)
F5

[{'Bread', 'Fruit', 'Jelly', 'PotatoChips', 'Soda'}]

Puedes revisar el algoritmo A-priori para la generación de conjuntos frecuentes en las siguientes imágenes.

<img src = "https://drive.google.com/uc?export=view&id=14cNXcjoDEaLVXLtXtTnq_FpyBxDO6-Gv" width="700"/>

<img src = "https://drive.google.com/uc?export=view&id=16kc4Apdel4x-HTeTD_VoAJfLMfO71ZK-" width="700"/>



# Generación de reglas

A partir de los conjuntos frecuentes podemos identificar reglas con suficiente **confianza**.

Para ejemplificar, consideremos un subconjunto de $F_4$

In [45]:
F4

[{'Bread', 'Fruit', 'Jelly', 'Milk'},
 {'Fruit', 'Milk', 'PeanutButter', 'Soda'},
 {'Fruit', 'Milk', 'PotatoChips', 'Soda'},
 {'Fruit', 'Jelly', 'Milk', 'PeanutButter'},
 {'Fruit', 'Jelly', 'Milk', 'Soda'},
 {'Bread', 'Fruit', 'Jelly', 'PotatoChips'},
 {'Bread', 'Fruit', 'PotatoChips', 'Soda'},
 {'Bread', 'Jelly', 'PotatoChips', 'Soda'},
 {'Fruit', 'Jelly', 'PotatoChips', 'Soda'},
 {'Bread', 'Fruit', 'Jelly', 'Soda'}]

Por ejemplo:

In [46]:
S = set(['Bread', 'Fruit', 'Jelly', 'Soda'])
S

{'Bread', 'Fruit', 'Jelly', 'Soda'}

**Identificamos, por fuerza bruta, todas las reglas que superen un valor mínimo de confianza, por ejemplo, 0.8**

Empecemos por formar reglas con un solo artículo en el consecuente:

In [47]:
conf_min = 0.8

# R1 
X = set(['Bread', 'Fruit', 'Jelly'])
Y = set(['Soda'])
if confianza(X,Y,T) > conf_min:
    print(str(X) + " ===> " + str(Y))

# R2
X = set(['Bread', 'Fruit', 'Soda'])
Y = set(['Jelly'])
if confianza(X,Y,T) > conf_min:
    print(str(X) + " ===> " + str(Y))

# R3
X = set(['Bread', 'Jelly', 'Soda'])
Y = set(['Fruit'])
if confianza(X,Y,T) > conf_min:
    print(str(X) + " ===> " + str(Y))

# R4
X = set(['Fruit', 'Jelly', 'Soda'])
Y = set(['Bread'])
if confianza(X,Y,T) > conf_min:
    print(str(X) + " ===> " + str(Y))
    

{'Bread', 'Fruit', 'Soda'} ===> {'Jelly'}
{'Bread', 'Jelly', 'Soda'} ===> {'Fruit'}


Probamos 4 reglas diferentes y solo 2 de ellas resultaron confiables.

**Busquemos reglas confiables con dos elementos en el consecuente**

In [48]:

# R1 
X = set(['Bread', 'Fruit'])
Y = set(['Jelly','Soda'])
if confianza(X,Y,T) > conf_min:
    print(str(X) + " ===> " + str(Y))

# R2
X = set(['Bread', 'Jelly'])
Y = set(['Fruit','Soda'])
if confianza(X,Y,T) > conf_min:
    print(str(X) + " ===> " + str(Y))

# R3
X = set(['Fruit', 'Jelly'])
Y = set(['Bread','Soda'])
if confianza(X,Y,T) > conf_min:
    print(str(X) + " ===> " + str(Y))

# R4
X = set(['Jelly', 'Soda'])
Y = set(['Bread','Fruit'])
if confianza(X,Y,T) > conf_min:
    print(str(X) + " ===> " + str(Y))
    
# R5
X = set(['Jelly', 'Fruit'])
Y = set(['Bread','Jelly'])
if confianza(X,Y,T) > conf_min:
    print(str(X) + " ===> " + str(Y))

# R6
X = set(['Bread', 'Soda'])
Y = set(['Fruit','Jelly'])
if confianza(X,Y,T) > conf_min:
    print(str(X) + " ===> " + str(Y))


{'Bread', 'Soda'} ===> {'Jelly', 'Fruit'}


Exploramos 6 reglas y una sola resultó confiable

Observa que **no es necesario explorar 6 reglas** con dos artículos en el consecuente, pues ninguna regla confiable puede tener 'Bread' o 'Soda' en el consecuente:

Por ejemplo, considera la regla que tiene $\{\text{Bread}\}$ como consecuente:

$$ \color{blue}{\{\text{Fruit}, \text{Jelly}, \text{Soda}\} → \{\text{Bread}\}}$$

Su confianza es:

$$\frac{\{\text{'Bread'}, \text{Fruit}, \text{Jelly}, \text{Soda}\}.\text{count}}{\{\text{Fruit}, \text{Jelly}, \text{Soda}\}.\text{count}} < \text{conf}_{\text{min}} $$

In [49]:
X = set(['Fruit','Jelly', 'Soda'])
Y = set(['Bread'])
confianza(X,Y,T)

0.75

Cualquier artículo que se agregue a $\{\text{Bread}\}$ al consecuente producirá una regla con menor o igual confianza:

Revisemos cada caso:

## Primera prueba:

$$ \color{blue}{\{\text{Jelly}, \text{Soda}\} → \{\text{Bread},\text{Fruit}\}}$$

Confianza

$$\frac{\{\text{Bread}, \text{Fruit}, \text{Jelly}, \text{Soda}\}.\text{count}}{\{\text{Jelly}, \text{Soda}\}.\text{count}}$$



In [50]:
X = set(['Jelly', 'Soda'])
Y = set(['Bread','Fruit'])
confianza(X,Y,T)

0.75

## Segunda prueba:

$$ \color{blue}{\{\text{Fruit}, \text{Soda}\} → \{\text{Bread},\text{Jelly}\}}$$

Confianza

$$\frac{\{\text{Bread}, \text{Fruit}, \text{Jelly}, \text{Soda}\}.\text{count}}{\{\text{Fruit}, \text{Soda}\}.\text{count}}$$



In [51]:
#title Texto de título predeterminado
X = set(['Fruit', 'Soda'])
Y = set(['Bread','Jelly'])
confianza(X,Y,T)

0.5

## Tercer prueba:

$$ \color{blue}{\{\text{Fruit}, \text{Jelly}\} → \{\text{Bread},\text{Soda}\}}$$

Confianza

$$\frac{\{\text{Bread}, \text{Fruit}, \text{Jelly}, \text{Soda}\}.\text{count}}{\{\text{Fruit}, \text{Jelly}\}.\text{count}}$$



In [52]:
X = set(['Fruit', 'Jelly'])
Y = set(['Bread','Soda'])
confianza(X,Y,T)

0.6

Recuerda que las únicas reglas confiables que identificamos son:

{'Bread', 'Soda', 'Fruit'} ===> {'Jelly'}

{'Bread', 'Soda', 'Jelly'} ===> {'Fruit'}

Dado que no podemos tener reglas confiables con 'Bread' o 'Soda' en el consecuente, la única regla con 2 artículos en el consecuente que debemos revisar es:

**{'Bread', 'Soda'} ===> {'Fruit', 'Jelly'}**

La cual es confiable:

In [53]:
# R6
X = set(['Bread', 'Soda'])
Y = set(['Fruit','Jelly'])
if confianza(X,Y,T) > conf_min:
    print(str(X) + " ===> " + str(Y))

{'Bread', 'Soda'} ===> {'Jelly', 'Fruit'}


Dado el razonamiento anterior podemos renunciar a explorar la formación de reglas con 3 artículos en el consecuente: **no es posible agregar 'Bread' o 'Fruit' al consecuente y obtener una regla confiable** 

El algoritmo para la generación de reglas se resume en la siguiente imagen:

<img src = "https://drive.google.com/uc?export=view&id=1kONlaT2zW-8qX41Lh5L5ENj5juhwmEVL" width="700"/>


# Reglas de asociación con Python

In [55]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [56]:
T

,Transaction
Customer,
1,"{Milk, PeanutButter, Bread, Fruit, Jelly}"
2,"{Milk, PeanutButter, PotatoChips, Bread, Fruit..."
3,"{WhippedCream, Fruit, Beer, ChocolateSauce}"
4,"{Steak, PotatoChips, Bread, Fruit, Jelly, Soda}"
5,"{Milk, PeanutButter, Fruit, Jelly, Soda}"
6,"{Milk, PotatoChips, Bread, Fruit, Jelly, Soda}"
7,"{PotatoChips, Fruit, Milk, Soda}"
8,"{Fruit, Milk, PeanutButter, Soda}"
9,"{Cheese, Fruit, Yogurt}"


In [57]:
te = TransactionEncoder()
te_ary = te.fit(T['Transaction']).transform(T['Transaction'])
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Beer,Bread,Cheese,ChocolateSauce,Fruit,Jelly,Milk,PeanutButter,PotatoChips,Soda,Steak,Vegetables,WhippedCream,Yogurt
0,False,True,False,False,True,True,True,True,False,False,False,False,False,False
1,False,True,False,False,True,True,True,True,True,True,False,True,False,False
2,True,False,False,True,True,False,False,False,False,False,False,False,True,False
3,False,True,False,False,True,True,False,False,True,True,True,False,False,False
4,False,False,False,False,True,True,True,True,False,True,False,False,False,False
5,False,True,False,False,True,True,True,False,True,True,False,False,False,False
6,False,False,False,False,True,False,True,False,True,True,False,False,False,False
7,False,False,False,False,True,False,True,True,False,True,False,False,False,False
8,False,False,True,False,True,False,False,False,False,False,False,False,False,True
9,True,False,False,False,False,False,False,False,False,False,False,True,False,True


In [58]:
#conjuntos frecuentes
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.2,(Beer)
1,0.4,(Bread)
2,0.9,(Fruit)
3,0.5,(Jelly)
4,0.6,(Milk)
...,...,...
85,0.2,"(Milk, PotatoChips, Bread, Fruit, Soda)"
86,0.2,"(Milk, PotatoChips, Bread, Jelly, Soda)"
87,0.2,"(Milk, PeanutButter, Fruit, Jelly, Soda)"
88,0.2,"(Milk, PotatoChips, Fruit, Jelly, Soda)"


In [59]:
from mlxtend.frequent_patterns import association_rules

In [60]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bread),(Fruit),0.4,0.9,0.4,1.00,1.111111,0.04,inf
1,(Bread),(Jelly),0.4,0.5,0.4,1.00,2.000000,0.20,inf
2,(Jelly),(Bread),0.5,0.4,0.4,0.80,2.000000,0.20,3.0
3,(Bread),(Milk),0.4,0.6,0.3,0.75,1.250000,0.06,1.6
4,(Bread),(PotatoChips),0.4,0.4,0.3,0.75,1.875000,0.14,2.4
...,...,...,...,...,...,...,...,...,...
281,"(Bread, Fruit, Milk, Soda)","(Jelly, PotatoChips)",0.2,0.3,0.2,1.00,3.333333,0.14,inf
282,"(Bread, Jelly, Milk, Soda)","(Fruit, PotatoChips)",0.2,0.4,0.2,1.00,2.500000,0.12,inf
283,"(Bread, PotatoChips, Milk)","(Jelly, Fruit, Soda)",0.2,0.4,0.2,1.00,2.500000,0.12,inf
284,"(Jelly, PotatoChips, Milk)","(Bread, Fruit, Soda)",0.2,0.3,0.2,1.00,3.333333,0.14,inf


<font color="red">Actividad </font>

El objetivo de esta práctica es el de explorar la aplicación del algoritmo Apriori en el análisis de la canasta de compras. Utilizarás el siguiente conjunto de datos disponible en el sitio Kaggle:

https://www.kaggle.com/irfanasrullah/groceries


1. Explora los datos (estructura, artículos, transacciones, ...). 
2. Identifica los artículos que con mayor frecuencia se presentan en las canastas de compras y muéstralos en una gráfica de barras (ordenados de acuerdo a la frecuencia de compra de forma descendente)
3. Explora la aplicación del algoritmo Apriori con diferentes soportes y valores de confianza mínimos.

In [61]:
data = pd.read_csv("./groceries - groceries.csv")
print(data.values.shape) 

(9835, 33)


In [62]:
data.head()

,Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,4,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
